In [ ]:
from source.source.table_utils import (
    collect_scores_into_dict_miss,
    extract_same_different_dataframes,
    aggregate_over_measures,
)
import pandas as pd
from IPython.display import display

pd.set_option("display.max_rows", None)

In [3]:
full_mis_rocauc = pd.read_csv("../../tables/full_mis_rocauc.csv", index_col=0)

In [ ]:
full_mis_rocauc = full_mis_rocauc[~full_mis_rocauc.UQMetric.str.endswith("Inner Inner")]

full_mis_rocauc.loc[
    (full_mis_rocauc.RiskType == "Bayes")
    & full_mis_rocauc.UQMetric.str.endswith("Outer"),
    "RiskType",
] = "Bayes Outer"
full_mis_rocauc.loc[
    (full_mis_rocauc.RiskType == "Bayes")
    & full_mis_rocauc.UQMetric.str.endswith("Inner"),
    "RiskType",
] = "Bayes Inner"

full_mis_rocauc.loc[
    (full_mis_rocauc.RiskType == "Total")
    & full_mis_rocauc.UQMetric.str.endswith("Outer"),
    "RiskType",
] = "Total Outer"
full_mis_rocauc.loc[
    (full_mis_rocauc.RiskType == "Total")
    & full_mis_rocauc.UQMetric.str.endswith("Inner"),
    "RiskType",
] = "Total Inner"

In [ ]:
# trunc_df = full_mis_rocauc[
# ~full_mis_rocauc.RiskType.isin(['Bias', 'MV', 'MVBI', 'BiasBI', 'Bregman Information', 'Reverse Bregman Information'])
# # full_mis_rocauc.base_rule.isin(['Brier', 'Logscore', 'Spherical']) &
# # full_mis_rocauc.LossFunction.isin(['Brier', 'Logscore', 'Spherical']) &
# # ~(np.isclose(full_mis_rocauc.RocAucScore, np.float64(0.5)))
# ]

# # trunc_df.sort_values(by='RocAucScore')

# trunc_df.to_csv(os.path.join('tables', 'full_mis_rocauc_only_risks.csv'), index=False)

In [ ]:
grouped_df = extract_same_different_dataframes(
    dataframe_=full_mis_rocauc,
)

In [ ]:
same_dict, _ = collect_scores_into_dict_miss(
    dataframes_list_=[
        grouped_df.logscore_logscore,
        grouped_df.brier_brier,
        grouped_df.spherical_spherical,
    ],
)
same_df = pd.DataFrame.from_dict(same_dict)

same_agg_df = aggregate_over_measures(
    dataframe_=same_df,
    agg_func_="mean",
    by_=["InD"],
)

In [ ]:
different_dict, _ = collect_scores_into_dict_miss(
    dataframes_list_=[
        grouped_df.logscore_not_logscore,
        grouped_df.brier_not_brier,
        grouped_df.spherical_not_spherical,
    ],
)
different_df = pd.DataFrame.from_dict(different_dict)

different_agg_df = aggregate_over_measures(
    dataframe_=different_df,
    agg_func_="mean",
    by_=["InD"],
)

In [ ]:
all_dict, _ = collect_scores_into_dict_miss(
    dataframes_list_=[
        full_mis_rocauc,
    ],
)
all_df = pd.DataFrame.from_dict(all_dict)

all_agg_df = aggregate_over_measures(
    dataframe_=all_df,
    agg_func_="mean",
    by_=["InD"],
)

In [ ]:
display(all_agg_df)
display(same_agg_df)
display(different_agg_df)

In [ ]:
same_agg_df.index

In [ ]:
def enhance_latex_table(input_latex):
    lines = input_latex.split("\n")
    enhanced_lines = []

    for i, line in enumerate(lines):
        if "\\toprule" in line:
            # Add multicolumn headers
            enhanced_lines.append(line)
            enhanced_lines.append(
                r"\multicolumn{2}{c}{Dataset} & \multicolumn{5}{c}{Metrics} \\"
            )
            enhanced_lines.append(r"\cmidrule(lr){1-2} \cmidrule(lr){3-7}")
            continue

        # Add row coloring
        if "\\midrule" in line:
            enhanced_lines.append(line)
            enhanced_lines.append(r"\rowcolor{gray!10}")
        elif "\\bottomrule" in line:
            enhanced_lines.append(r"\end{tabular}")
        else:
            enhanced_lines.append(line)

    return "\n".join(enhanced_lines)


def get_nice_df(df_):
    df_.index = pd.Index(
        data=[
            "CIFAR10",
            "CIFAR100",
            "Missed class CIFAR10",
            "Noisy CIFAR10",
            "Noisy CIFAR100",
        ],
        name="InD",
    )
    df_.columns = [
        # 'Bayes',
        # 'Excess',
        # 'Total',
        "Bayes(O)",
        "Bayes(I)",
        "Total(O)",
        "Total(I)",
        "BI",
        "RBI",
        "EPBI",
        # 'Bias',
        # 'MV',
        # 'MVBI',
        # 'BiasBI',
    ]
    # df_ = df_[['Bayes', 'Excess', 'Total', 'BI', 'RBI']]
    df_ = (100 * df_).round(2)

    display(df_)

    return df_, df_.to_latex(float_format="%.2f")

In [ ]:
measures = [
    "Bayes Outer",
    "Bayes Inner",
    "Total Outer",
    "Total Inner",
    "Bregman Information",
    "Reverse Bregman Information",
    "Expected Pairwise Bregman Information",
]

# measures = ['Bayes', 'Excess', 'Total', 'Bregman Information', 'Reverse Bregman Information', 'Expected Pairwise Bregman Information']

In [ ]:
nice_same = get_nice_df(same_agg_df.copy()[measures])
print(enhance_latex_table(nice_same[1]))

In [ ]:
nice_different = get_nice_df(different_agg_df.copy()[measures])
print(enhance_latex_table(nice_different[1]))

In [ ]:
print(
    enhance_latex_table(
        pd.concat([nice_same[0], nice_different[0]], axis=1).to_latex(
            float_format="%.2f"
        )
    )
)

In [ ]:
nice_same[0].mean()

In [ ]:
nice_same[0].std()

In [ ]:
nice_different[0].mean()

In [ ]:
nice_different[0].std()

In [ ]:
same_agg_df.eq(same_agg_df.max(axis=1), axis=0)

In [ ]:
different_agg_df.eq(different_agg_df.max(axis=1), axis=0)

In [ ]:
all_agg_df.eq(all_agg_df.max(axis=1), axis=0)

In [ ]:
(same_agg_df - different_agg_df) > 0